In [1]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sqlalchemy
import h5py

In [2]:
db_connection_string = 'sqlite:///./Resources/product.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['AAPL_1_Day_Candles', 'AAPL_1_Min_Candles', 'AAPL_Indicators', 'AAPL_Info']


In [4]:
ticker = 'AAPL'
indicators_df = pd.read_sql_table(ticker + '_Indicators', con=engine, index_col='Datetime')

---

## Prepare the data to be used on a neural network model

In [5]:
print(indicators_df.head())

                        Open     High      Low    Close  Volume  CDLBELTHOLD  \
Datetime                                                                       
2021-09-03 14:26:00  154.355  154.355  154.245  154.250  158378            0   
2021-09-03 14:27:00  154.250  154.380  154.250  154.300  239650            0   
2021-09-03 14:28:00  154.305  154.310  154.270  154.285   55602            0   
2021-09-03 14:29:00  154.280  154.285  154.230  154.245   79606            0   
2021-09-03 14:30:00  154.250  154.290  154.240  154.280  100991            0   

                     CDLBREAKAWAY  CDLDOJISTAR  CDLLADDERBOTTOM  Trade Signal  
Datetime                                                                       
2021-09-03 14:26:00             0            0                0           0.0  
2021-09-03 14:27:00             0            0                0           0.0  
2021-09-03 14:28:00             0            0                0           0.0  
2021-09-03 14:29:00             0      

In [6]:
# Review the data types associated with the columns
indicators_df.dtypes

Open               float64
High               float64
Low                float64
Close              float64
Volume               int64
CDLBELTHOLD          int64
CDLBREAKAWAY         int64
CDLDOJISTAR          int64
CDLLADDERBOTTOM      int64
Trade Signal       float64
dtype: object

In [7]:
display(indicators_df['Trade Signal'].value_counts())

 0.0    10956
 1.0     1957
-1.0     1924
Name: Trade Signal, dtype: int64

### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [8]:
# Create a list of categorical variables 
categorical_variables = ['Trade Signal']

# Display the categorical variables list
display(categorical_variables)

['Trade Signal']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [10]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(indicators_df[categorical_variables])


In [11]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)
# Review the DataFrame
display(encoded_df.head(10))

,Trade Signal_-1.0,Trade Signal_0.0,Trade Signal_1.0
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
5,0.0,1.0,0.0
6,0.0,1.0,0.0
7,0.0,1.0,0.0
8,0.0,1.0,0.0
9,0.0,1.0,0.0


In [12]:
encoded_df.rename(columns={'Trade Signal_-1.0': 'Bearish', 'Trade Signal_0.0': 'None', 'Trade Signal_1.0':'Bullsih'}, inplace=True)
encoded_df.drop(columns='None', inplace=True)
print(encoded_df.head())

   Bearish  Bullsih
0      0.0      0.0
1      0.0      0.0
2      0.0      0.0
3      0.0      0.0
4      0.0      0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [13]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df.copy()

# Display a sample of y
display(y.head())

,Bearish,Bullsih
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [14]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = indicators_df.drop(columns=['Trade Signal'])

# Review the features DataFrame
display(X.head())

,Open,High,Low,Close,Volume,CDLBELTHOLD,CDLBREAKAWAY,CDLDOJISTAR,CDLLADDERBOTTOM
Datetime,,,,,,,,,
2021-09-03 14:26:00,154.355,154.355,154.245,154.250,158378,0,0,0,0
2021-09-03 14:27:00,154.250,154.380,154.250,154.300,239650,0,0,0,0
2021-09-03 14:28:00,154.305,154.310,154.270,154.285,55602,0,0,0,0
2021-09-03 14:29:00,154.280,154.285,154.230,154.245,79606,0,0,0,0
2021-09-03 14:30:00,154.250,154.290,154.240,154.280,100991,0,0,0,0


### Step 6: Split the features and target sets into training and testing datasets.


In [15]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [17]:
# Define the the number of inputs (features) to the model
number_input_features = len(list(X.columns))
# Review the number of features
number_input_features


9

In [18]:
# Define the number of neurons in the output layer
number_output_neurons = 2

In [19]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  int(round((number_input_features + number_output_neurons)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1


6

In [20]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  int(round((hidden_nodes_layer1 + number_output_neurons)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


4

In [21]:
# Create the Sequential model instance
nn = Sequential()


In [22]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))


In [23]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [24]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [25]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 60        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
Total params: 98
Trainable params: 98
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [26]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [27]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [28]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

116/116 [==============================] - 0s 375us/step - loss: 8.3040e-08 - accuracy: 0.9973
Loss: 8.303999976533305e-08, Accuracy: 0.9973045587539673


### Step 4: Save and export your model to an HDF5 file

In [29]:
# Set the model's file path
filename = "DLNN_model_1"
file_path = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn.save(file_path)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1 - Deep learning model with 2 hidden layers

In [30]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


9


In [31]:
# Review the number of features
number_input_features

9

In [32]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 2

In [33]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 =  int(round((number_input_features + number_output_neurons_A1)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A1


6

In [34]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  int(round((hidden_nodes_layer1_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A1


4

In [35]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A1 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A1


3

In [36]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [37]:
# Add the first hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation="relu", input_dim=number_input_features))


In [38]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))


In [39]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer3_A1, activation="relu"))


In [40]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))


In [41]:
# Display the Sequential model summary
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 60        
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 8         
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [43]:
# Fit the model using 50 epochs and the training data
model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Alternative Model 2 - Deep learning model with 3 hidden layers

In [44]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


9


In [45]:
# Review the number of features
number_input_features

9

In [46]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 2

In [47]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 =  int(round((number_input_features + number_output_neurons_A2)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A2


6

In [48]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  int(round((hidden_nodes_layer1_A2 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2


4

In [49]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A2 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A2


3

In [50]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer4_A2 =  int(round((hidden_nodes_layer3_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer4_A2


2

In [51]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [52]:
# Add the first hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu", input_dim=number_input_features))


In [53]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))


In [54]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu"))


In [55]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2, activation="relu"))


In [56]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))


In [57]:
# Display the Sequential model summary
nn_A2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 6)                 60        
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 6         
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [58]:
# Compile the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [59]:
# Fit the model using 50 epochs and the training data
model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50, verbose=0)


#### Alternative Model 3 - Principal Component Analysis

In [60]:
from sklearn.decomposition import PCA


In [61]:
# Create a StandardScaler instance
pca_scaler = StandardScaler()

# Fit the scaler to the features
X_pca_scaler = pca_scaler.fit(X)

# Fit the scaler to the features
X_pca_scaled = X_pca_scaler.transform(X)


In [68]:
# for n in range(2, number_input_features):
best_n = number_input_features
for n in range(number_input_features, 2, -1):
    pca = PCA(n_components=n, random_state=1)
    X_pca = pca.fit_transform(X_pca_scaled)
    exp_var = pca.explained_variance_ratio_
#     print(f'{exp_var}')
    print(f'N = {n:3}: Total explained variance: {100*exp_var.sum():.4f}%')
    if exp_var.sum() > .999:
        best_n = n

N =   9: Total explained variance: 100.0000%
N =   8: Total explained variance: 100.0000%
N =   7: Total explained variance: 99.9998%
N =   6: Total explained variance: 99.9987%
N =   5: Total explained variance: 99.9968%
N =   4: Total explained variance: 87.5497%
N =   3: Total explained variance: 75.0507%


In [69]:
print(best_n)
pca = PCA(n_components=best_n, random_state=1)
X_pca = pca.fit_transform(X_pca_scaled)
exp_var = pca.explained_variance_ratio_
print(f'N = {best_n:3}: Total explained variance: {100*exp_var.sum():.4f}%')


9
N =   9: Total explained variance: 100.0000%


In [ ]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, random_state=1)

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = best_n
print(number_input_features)


13


In [ ]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 2

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 =  int(round((number_input_features + number_output_neurons_A3)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A3


8

In [ ]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 =  int(round((hidden_nodes_layer1_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A3


5

In [ ]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A3 =  int(round((hidden_nodes_layer2_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the third layer
hidden_nodes_layer3_A3


4

In [ ]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [ ]:
# Add the first hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, activation="relu", input_dim=number_input_features))


In [ ]:
# Add the second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))


In [ ]:
# Add the third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))


In [ ]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A3.add(Dense(units=number_output_neurons_A3, activation="sigmoid"))


In [ ]:
# Display the Sequential model summary
nn_A3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 24        
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 10        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the Sequential model
nn_A3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Fit the model using 50 epochs and the training data
model_A3 = nn_A3.fit(X_pca_train, y_pca_train, epochs=50, verbose=1)


Epoch 1/50
293/293 [==============================] - 0s 476us/step - loss: 0.3251 - accuracy: 0.8975
Epoch 2/50
293/293 [==============================] - 0s 475us/step - loss: 0.1694 - accuracy: 0.9153
Epoch 3/50
293/293 [==============================] - 0s 465us/step - loss: 0.1578 - accuracy: 0.9202
Epoch 4/50
293/293 [==============================] - 0s 471us/step - loss: 0.1544 - accuracy: 0.9206
Epoch 5/50
293/293 [==============================] - 0s 465us/step - loss: 0.1524 - accuracy: 0.9197
Epoch 6/50
293/293 [==============================] - 0s 465us/step - loss: 0.1508 - accuracy: 0.9184
Epoch 7/50
293/293 [==============================] - 0s 468us/step - loss: 0.1485 - accuracy: 0.9160
Epoch 8/50
293/293 [==============================] - 0s 488us/step - loss: 0.1466 - accuracy: 0.9136
Epoch 9/50
293/293 [==============================] - 0s 505us/step - loss: 0.1451 - accuracy: 0.9136
Epoch 10/50
293/293 [==============================] - 0s 472us/step - loss: 0.143

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [ ]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
98/98 [==============================] - 0s 397us/step - loss: 0.1395 - accuracy: 0.9089
Loss: 0.13953788578510284, Accuracy: 0.9089452028274536


In [ ]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
98/98 [==============================] - 0s 391us/step - loss: 0.1365 - accuracy: 0.9093
Loss: 0.13650137186050415, Accuracy: 0.9092658162117004


In [ ]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
98/98 [==============================] - 0s 385us/step - loss: 0.1399 - accuracy: 0.9093
Loss: 0.13988523185253143, Accuracy: 0.9092658162117004


In [ ]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_pca_test, y_pca_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
98/98 [==============================] - 0s 380us/step - loss: 0.1425 - accuracy: 0.9105
Loss: 0.14249324798583984, Accuracy: 0.9105482697486877


### Run different numbers of epochs on best model

In [ ]:
# Fit the model using the training data with different numbers of epochs
model_list = []
for n in range(50, 151, 20):
    print(f'Fitting model, epochs: {n}')
    model_list.append(nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False))
#     model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False)

    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
    model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

Fitting model, epochs: 50
98/98 [==============================] - 0s 391us/step - loss: 0.1350 - accuracy: 0.9093
Loss: 0.13496015965938568, Accuracy: 0.9092658162117004
Fitting model, epochs: 70
98/98 [==============================] - 0s 386us/step - loss: 0.1352 - accuracy: 0.9089
Loss: 0.13516975939273834, Accuracy: 0.9089452028274536
Fitting model, epochs: 90
98/98 [==============================] - 0s 381us/step - loss: 0.1346 - accuracy: 0.9093
Loss: 0.13464869558811188, Accuracy: 0.9092658162117004
Fitting model, epochs: 110
98/98 [==============================] - 0s 412us/step - loss: 0.1351 - accuracy: 0.9083
Loss: 0.13511475920677185, Accuracy: 0.9083039164543152
Fitting model, epochs: 130
98/98 [==============================] - 0s 404us/step - loss: 0.1358 - accuracy: 0.9089
Loss: 0.13577121496200562, Accuracy: 0.9089452028274536
Fitting model, epochs: 150
98/98 [==============================] - 0s 418us/step - loss: 0.1373 - accuracy: 0.9086
Loss: 0.13730131089687347, 

In [ ]:
# for model in model_list:
#     pass
#     print(model.history)
    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
#     model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
#     print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

### Step 3: Save each of your alternative models as an HDF5 file.


In [ ]:
# Set the file path for the first alternative model
filename = "DLNN_model_A1"
file_path_A1 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A1.save(file_path_A1)


In [ ]:
# Set the file path for the second alternative model
filename = "DLNN_model_A2"
file_path_A2 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path_A2)


In [ ]:
# # Set the file path for the third alternative model
filename = "DLNN_model_A3"
file_path_A3 = Path('Resources/' + filename + '.h5')

# # Export your model to a HDF5 file
nn_A3.save(file_path_A3)
